In [ ]:
import glob
import os

def get_all_aligned_spectrograms(base_dir):
    """
    Search recursively under base_dir for files of the form 
    aligned_spectrogram_<n>.png. 
    Returns a list of all matches sorted first by folder, 
    then numerically by <n>.
    """
    pattern = os.path.join(base_dir, "*", "aligned_spectrogram_*.png")
    files = glob.glob(pattern)

    def sort_key(path):
        folder = os.path.dirname(path)
        idx = int(os.path.splitext(path)[0].split('_')[-1])
        return (folder, idx)

    files.sort(key=sort_key)
    return files


# Example usage:
base_dir = "../Results/Dev"
filenames = get_all_aligned_spectrograms(base_dir)

print(f"Found {len(filenames)} files")
print(filenames[:20])  # show first 20


In [ ]:
# pixel_pca_incremental.py
import os
from PIL import Image
import numpy as np
from sklearn.decomposition import IncrementalPCA
from tqdm.auto import tqdm

def image_paths_from_list(filenames):
    return filenames  # assume filenames is a list you already generated

def build_memmap(image_paths, memmap_path, img_size=(128, 128), dtype=np.float32):
    """
    Create a memmap of shape (N, D) where D = H*W, grayscale and flattened.
    """
    N = len(image_paths)
    H, W = img_size
    D = H * W
    os.makedirs(os.path.dirname(memmap_path), exist_ok=True)
    mm = np.memmap(memmap_path, dtype=dtype, mode='w+', shape=(N, D))
    for i, p in enumerate(tqdm(image_paths, desc="Writing memmap")):
        img = Image.open(p).convert('L').resize((W, H), Image.BILINEAR)
        arr = np.asarray(img, dtype=dtype).reshape(-1) / 255.0
        mm[i, :] = arr
    mm.flush()
    return memmap_path, (N, D)

def run_incremental_pca(memmap_path, shape, n_components=100, batch_size=64, save_path="pixel_pca.npz"):
    N, D = shape
    mm = np.memmap(memmap_path, dtype=np.float32, mode='r', shape=(N, D))

    ipca = IncrementalPCA(n_components=n_components)
    # First pass: fit in batches to compute principal components
    for start in tqdm(range(0, N, batch_size), desc="IPCA fit"):
        batch = mm[start:start+batch_size]
        ipca.partial_fit(batch)
    # Optionally transform all data (second pass)
    components = ipca.components_          # shape (n_components, D)
    mean = ipca.mean_                      # shape (D,)
    explained = ipca.explained_variance_ratio_

    np.savez_compressed(save_path,
                        components=components,
                        mean=mean,
                        explained=explained,
                        n_components=n_components)
    print(f"Saved PCA data to {save_path}")
    return ipca

# Example usage
if __name__ == "__main__":
    # assume filenames is a list of filepaths (numeric-sorted)
    from glob import glob
    memmap_path = "../Results/EmbeddingData/pixel_memmap.dat"
    memmap_path, shape = build_memmap(filenames, memmap_path, img_size=(128,128))
    ipca = run_incremental_pca(memmap_path, shape, n_components=50, batch_size=64, save_path="../Results/EmbeddingData/pixel_pca.npz")


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

data = np.load("../Results/EmbeddingData/pixel_pca.npz")
comps = data["components"]  # (k, D)
H, W = 128, 128
k = 10
fig, axs = plt.subplots(1, k, figsize=(15,3))
for i in range(k):
    face = comps[i].reshape(H, W)
    axs[i].imshow((face - face.min())/(face.max()-face.min()), cmap='gray')
    axs[i].axis('off')
plt.show()


In [ ]:
import glob
import os

def get_aligned_spectrograms(prefix):
    """
    Given a prefix like '../Results/Dev/rock.00069/aligned_spectrogram',
    return all matching files of the form aligned_spectrogram_<n>.png,
    sorted numerically by <n>.
    """
    pattern = f"{prefix}_*.png"
    files = glob.glob(pattern)
    files.sort(key=lambda x: int(os.path.splitext(x)[0].split('_')[-1]))
    return files


# Example usage:
prefix = "../Results/Dev/rock.00069/aligned_spectrogram"
filenames = get_aligned_spectrograms(prefix)

print(filenames)

In [ ]:
# pixel_pca_inference_matrix.py
import os
import numpy as np
import torch
from PIL import Image
from tqdm.auto import tqdm

def load_pca(npz_path):
    """Reload trained PCA from .npz file."""
    data = np.load(npz_path)
    components = data["components"]      # (n_components, D)
    mean = data["mean"]                  # (D,)
    n_components = int(data["n_components"])
    return components, mean, n_components

def project_images_matrix(filenames, pca_npz_path, save_path="pixel_embeddings.pt", img_size=(128,128)):
    H, W = img_size

    # Load PCA weights
    components, mean, n_components = load_pca(pca_npz_path)

    all_embeds = []

    for fn in tqdm(filenames, desc="Projecting images"):
        # Load + preprocess image
        img = Image.open(fn).convert("L").resize((W, H), Image.BILINEAR)
        arr = np.asarray(img, dtype=np.float32).reshape(-1) / 255.0

        # Center and project
        arr_centered = arr - mean
        embed = np.dot(components, arr_centered)   # shape (n_components,)

        all_embeds.append(embed)

    # Stack into (N, n_components)
    embeddings_tensor = torch.tensor(np.stack(all_embeds), dtype=torch.float32)

    # Save tensor
    torch.save(embeddings_tensor, save_path)
    print(f"Saved embeddings to {save_path}, shape={embeddings_tensor.shape}")

    return embeddings_tensor



embeddings = project_images_matrix(
    filenames,
    pca_npz_path="../Results/EmbeddingData/pixel_pca.npz",
    img_size=(128,128)
)
print(embeddings.shape)  # e.g. torch.Size([27, 512])


In [ ]:
type(embeddings)